# Изображения

In [6]:
import imageio.v3 as iio

img_arr = iio.imread('/home/avtotka/Pictures/hurrem.jpg')
img_arr.shape # ширина * высота * каналы

(1280, 960, 3)

In [9]:
import torch

img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1) # перестановка на каналы * высота * ширина (стандарт для PyTorch)
out.shape

torch.Size([3, 1280, 960])

Для более эффективной загрузки нескольких изображений - выделить память заранее под тензор нужного размера

In [15]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 1280, 960, dtype=torch.uint8)
batch.shape

torch.Size([3, 3, 1280, 960])

In [17]:
import os

data_dir = '/home/avtotka/Pictures/960x1280'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.jpg']
for i, filename in enumerate(filenames):
    img_arr = iio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3] # убираю альфа канал если есть
    batch[i] = img_t

## Нормализация данных

In [21]:
batch = batch.float()

n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

batch

tensor([[[[ 0.9638,  0.7653,  0.6931,  ...,  1.3427,  1.1984,  0.9999],
          [ 0.9999,  0.9457,  0.7833,  ...,  1.2344,  1.1803,  1.0720],
          [ 1.1262,  1.1803,  0.9638,  ...,  1.1262,  1.1442,  1.1081],
          ...,
          [ 0.2239,  0.2419,  0.2780,  ...,  0.1698,  0.0976,  0.0434],
          [ 0.2600,  0.3141,  0.3502,  ...,  0.0976,  0.0976,  0.1156],
          [ 0.3322,  0.3863,  0.4404,  ...,  0.0434,  0.0976,  0.1698]],

         [[ 0.7973,  0.6090,  0.5336,  ...,  1.2494,  1.0987,  0.9104],
          [ 0.8350,  0.7973,  0.6278,  ...,  1.1552,  1.1176,  1.0045],
          [ 0.9857,  1.0422,  0.8162,  ...,  1.0799,  1.0987,  1.0610],
          ...,
          [-0.2952, -0.2763, -0.2387,  ..., -0.3329, -0.4082, -0.4647],
          [-0.2575, -0.2010, -0.1633,  ..., -0.4082, -0.4082, -0.3894],
          [-0.1822, -0.1257, -0.0691,  ..., -0.4647, -0.4082, -0.3329]],

         [[-0.6616, -0.8720, -0.9367,  ..., -0.5807, -0.7102, -0.9206],
          [-0.6292, -0.7102, -

# Таблицы

In [25]:
import csv
import numpy as np

wine_path = '/home/avtotka/dev/ai/pytorch/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]],
      shape=(4898, 12), dtype=float32)

In [26]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [27]:
wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [28]:
data = wineq[:, :-1] # все строки и столбцы кроме последнего (без quality)
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [29]:
target = wineq[:, -1] # только оценки
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [45]:
target = target.long()
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

target_onehot

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [73]:
index = torch.tensor([[0, 1, 2]])
value = torch.tensor([[4., 5., 6.]])
torch.zeros(3, 5).scatter_(0, index, value)

tensor([[4., 0., 0., 0., 0.],
        [0., 5., 0., 0., 0.],
        [0., 0., 6., 0., 0.]])